**MSc Computational Physics AUTH**<br>
**Quantum Mechanics**<br>
**Academic Year: 2024-2025**<br>
**Semester 3**<br>
**Implemented by: Ioannis Stergakis**<br>
**AEM: 4439**<br>

**Presentation date: 7/1/2025**

# **PROJECT:** ***Calculating dot products of states in the hydrogen atom***<br>

**Dot products to be calculated:**<br>
*a) $<210|x|311>$<br>
b) $<211|x|310>$<br>
c) $<210|y|311>$<br>
d) $<211|y|310>$*

**Contents:**<br>
*->1. Defining and displaying the requested states in hydrogeneous atoms<br>
->2. Calculating the requested dot products*

## 1. Defining and displaying the requested states in hydrogeneous atoms

### **1.1 Preliminaries**

In [1]:
import numpy as np
import sympy as smp
from scipy.integrate import quad,dblquad,tplquad
from scipy.special import sph_harm,genlaguerre
import math

In [2]:
# Useful symbols definition
r,theta,phi,Zeta,alpha0 = smp.symbols("r,θ,φ,Z,α_0",real=True)
i = smp.symbols("i")
i_star = smp.conjugate(i)

# Useful constants definition
a_0 = 0.0529 # Bohr radius in nm

# Useful functions definition
def x(r,θ,φ): # x-axis coordinate in spherical coordinates
    return r*np.sin(θ)*np.cos(φ)

def y(r,θ,φ): # x-axis coordinate in spherical coordinates
    return r*np.sin(θ)*np.sin(φ)

### **1.2 Defining the radial part $R_{nl}(r)$ of the hydrogeneous states $u_{nlm}(r,θ,φ)$**

The formula of $R_{nl}(r)$ for hydrogeneous atoms is:
$$R_{nl}(r)=N_{nl}\left(\frac{2Z}{na_0}r\right)^l L^{2l+1}_{n-l-1}(\frac{2Z}{na_0}r)e^{-\frac{Zr}{na_0}}\space\space[1.1]$$
where $Z$ is the atomic number and $N_{nl}$ is the **normalization factor**:
$$N_{nl}=\left[\left(\frac{2Z}{na_0}\right)^3\frac{(n-l-1)!}{2n(n+l)!}\right]^{1/2}\space\space[1.2]$$
obtained from the integral:
$$\int^{\infty}_0 R^{2}_{nl}(r)r^2 dr=1,$$
the $L^{ν}_k(ξ)$ is the generalized associated **Laguerre** polynomial:
$$L^{ν}_k(ξ)=\frac{1}{k!}ξ^{-ν}e^ξ\frac{d^k}{dξ^k}(e^{-ξ}ξ^{k+ν})\space\space[1.3]$$
and $a_0$ is the **Bohr** radius:
$$a_0=\frac{\hbar^2}{μe^2}$$
or
$$a_0=0.0529 nm$$

First, we define the normalization factor $N_{nl}$ as a function of the quantum numbers $n$ and $l$ and the atomic number $Z$. This is shown below:

In [3]:
# Numerical definition of the normalization factor
def N_nl(n,l,Z):
    return smp.sqrt((2*Z/(n*a_0))**3*math.factorial(n-l-1)/(2*n*math.factorial(n+l)))

# Symbolic definition of the normalization factor
def N_nl_sym(n,l,Z):
    return smp.sqrt((2*Z/(n*alpha0))**3*math.factorial(n-l-1)/(2*n*math.factorial(n+l)))

Next, we define generalized associated **Laguerre** polynomial. We start with the symbolic definition. From the equations $[1.1]$ and $[1.3]$, we obtain the following relations for the indices $\nu$ and $l$:
$$\nu=2l+1\space\space[1.4]$$ 
and 
$$k=n-l-1\space\space[1.5]$$

In our study, we have either the combination of $n=2$ and $l=1$ or the combination $n=3$ and $l=1$. That is, we have the combination:
$$\nu=3,k=0$$
or the combination:
$$\nu=3,k=1$$
for the indices $\nu$ and $k$, respectively. For the first case ($\nu=3,k=0$) the **Laguerre** polynomial becomes, according to $[1.3]$:
$$L^{ν}_k(ξ)=L^{3}_0(ξ)=\frac{1}{0!}ξ^{-3}e^ξ\frac{d^0}{dξ^0}(e^{-ξ}ξ^{0+3})$$
$$L^{3}_0(ξ)=ξ^{-3}e^ξ*(e^{-ξ}ξ^{3})$$
$$L^{3}_0(ξ)=1\space\space[1.6]$$
For the second case ($\nu=3,k=1$) the **Laguerre** polynomial becomes:
$$L^{ν}_k(ξ)=L^{3}_1(ξ)=\frac{1}{1!}ξ^{-3}e^ξ\frac{d}{dξ}(e^{-ξ}ξ^{1+3})$$
$$L^{3}_1(ξ)=ξ^{-3}e^ξ\frac{d}{dξ}(e^{-ξ}ξ^{4})$$
$$L^{3}_1(ξ)=ξ^{-3}e^ξ*(-e^{-ξ}ξ^{4}+4*e^{-ξ}ξ^{3})$$
$$L^{3}_1(ξ)=4-ξ\space\space[1.7]$$

The symbolic definition is shown below:

In [4]:
# Symbolic definition of the necessary generalized associated Laquerre polynomials

# Polynomial for n=2 and l=1
# or ν = 2l+1 = 3 and k = n-l-1 = 0
def L_30_sym(ξ):
    return 1

# Polynomial for n=3 and l=1
# or ν = 2l+1 = 3 and k = n-l-1 = 1
def L_31_sym(ξ):
    return 4-ξ

# Storaging the Laquerre polynomials in a matrix
def L_vk_list_sym(ξ):
    result = []
    result.append(L_30_sym(ξ))
    result.append(L_31_sym(ξ))
    return result

As for the numerical definition of the **Laguerre** polynomial, we use the imported ***genlaguerre*** function of the ***scipy.special*** class, inside the numerical definition of the whole radial wave-function $R_{nl}(r)$:

In [5]:
# Numerical definition of the radial part R_nl(r)
def R_nl(r,n,l,Z):
    return N_nl(n,l,Z)*(2*Z*r/(n*a_0))**l*genlaguerre(n-l-1,2*l+1)(2*Z*r/(n*a_0))*np.exp(-Z*r/(n*a_0))
    

# Symbolic definition of the radial part R_nl(r)
def R_nl_sym(r,n,l,Z):
    L_vk = L_vk_list_sym(2*Z*r/(n*alpha0))[n-2]
    return N_nl_sym(n,l,Z)*(2*Z*r/(n*alpha0))**l*L_vk*smp.exp(-Z*r/(n*alpha0))

In [6]:
# Symbolic definitions of the necessary radial functions

# Definition of R_nl = R_21(r) radial function
def R_21_sym(r,Z):
    return R_nl_sym(r,2,1,Z)

# Printing the formula of R_21(r)
print("R_21(r,Z)")
display(R_21_sym(r,Zeta))

# Definition of R_nl = R_31(r) radial function
def R_31_sym(r,Z):
    return R_nl_sym(r,3,1,Z)

# Printing the formula of R_21(r)
print("R_31(r,Z)")
display(R_31_sym(r,Zeta))

R_21(r,Z)


sqrt(6)*Z*r*sqrt(Z**3/α_0**3)*exp(-Z*r/(2*α_0))/(12*α_0)

R_31(r,Z)


sqrt(6)*Z*r*sqrt(Z**3/α_0**3)*(-2*Z*r/(3*α_0) + 4)*exp(-Z*r/(3*α_0))/(81*α_0)

In [7]:
# Numerical definitions of the necessary radial functions

# Definition of R_nl = R_21(r) radial function
def R_21(r,Z):
    return R_nl(r,2,1,Z)

# Definition of R_nl = R_31(r) radial function
def R_31(r,Z):
    return R_nl(r,3,1,Z)

### **1.3 Defining the necessary spherical harmonics functions $Y^m_l(θ,φ)$**

For our study, we need the spherical harmonic functions for $l=1$ and $m=0,1$. These are the following:
$$Y^0_1=\sqrt{\frac{3}{4\pi}}\cos{\theta}$$
$$Y^1_1=-\sqrt{\frac{3}{8\pi}}\sin{\theta}e^{i\phi}$$

Their symbolic definition comes first:

In [8]:
# Symbolic definitions of the necessary spherical harmonics

# Definition of the Y_lm = Y_10(θ,φ) spherical harmonic
def Y_10_sym(θ,φ):
    return smp.sqrt(3/(4*smp.pi))*smp.cos(θ)

# Definition of the Y_lm = Y_11(θ,φ) spherical harmonic
def Y_11_sym(θ,φ):
    return -smp.sqrt(3/(8*smp.pi))*smp.sin(θ)*smp.exp(1j*φ)

# Definition of the conjugate of Y_11
def Y_10_star_sym(θ,φ):
    result = smp.conjugate(Y_10_sym(θ,φ))
    return result

# Definition of the conjugate of Y_11
def Y_11_star_sym(θ,φ):
    result = smp.conjugate(Y_11_sym(θ,φ))
    return result

and is followed by their numerical definition, as shown below:

In [9]:
# Numerical definitions of the necessary spherical harmonics

# Definition of the Y_lm = Y_10(θ,φ) spherical harmonic
def Y_10(θ,φ):
    return sph_harm(0,1,φ,θ)

# Definition of the Y_lm = Y_11(θ,φ) spherical harmonic
def Y_11(θ,φ):
    return sph_harm(1,1,φ,θ)

See that we used the imported ***sph_harm*** function of the ***scipy.special*** class.

### **1.4 Showing the requested states of hydrogeneous atoms and their conjugates**

Having defined all the separate components (radial and angular), we proceed with the definition and display of the states $u_{nlm}(r,θ,φ)$, as well as their respective conjugates, that take part in the requested dot products.

First, we have the state **|210>**$=u_{210}(r,θ,φ,Ζ)=R_{21}(r,Ζ)*Y^0_1(θ,φ)$:

In [10]:
# State u_210 = R_21*Y_10 of hydrogeneous atoms

# Numerical definition
def u_210(r,θ,φ,Z):
    return R_21(r,Z)*Y_10(θ,φ)

# Symbolic definitions
def u_210_sym(r,θ,φ,Z):
    return R_nl_sym(r,2,1,Z)*Y_10_sym(θ,φ)

def u_210_star_sym(r,θ,φ,Z):
    return R_nl_sym(r,2,1,Z)*Y_10_star_sym(θ,φ)

# Printing formula of the state
print("u_210(r,θ,φ,Z)")
display(u_210_sym(r,theta,phi,Zeta))

# Printing formula of the state's conjugate
print("u_210(r,θ,φ,Z)*")
display(u_210_star_sym(r,theta,phi,Zeta))

u_210(r,θ,φ,Z)


sqrt(2)*Z*r*sqrt(Z**3/α_0**3)*exp(-Z*r/(2*α_0))*cos(θ)/(8*sqrt(pi)*α_0)

u_210(r,θ,φ,Z)*


sqrt(2)*Z*r*sqrt(Z**3/α_0**3)*exp(-Z*r/(2*α_0))*cos(θ)/(8*sqrt(pi)*α_0)

Second, we have the state **|211>**$=u_{211}(r,θ,φ,Ζ)=R_{21}(r,Ζ)*Y^1_1(θ,φ)$:

In [11]:
# State u_211 = R_21*Y_11 of hydrogeneous atoms

# Numerical definition
def u_211(r,θ,φ,Z):
    return R_21(r,Z)*Y_11(θ,φ)

# Symbolic definitions
def u_211_sym(r,θ,φ,Z):
    return R_nl_sym(r,2,1,Z)*Y_11_sym(θ,φ)

def u_211_star_sym(r,θ,φ,Z):
    return R_nl_sym(r,2,1,Z)*Y_11_star_sym(θ,φ)

# Printing formula of the state
print("u_211(r,θ,φ,Z)")
display(u_211_sym(r,theta,phi,Zeta))

# Printing formula of the state's conjugate
print("u_211(r,θ,φ,Z)*")
display(u_211_star_sym(r,theta,phi,Zeta))

u_211(r,θ,φ,Z)


-Z*r*sqrt(Z**3/α_0**3)*exp(1.0*I*φ)*exp(-Z*r/(2*α_0))*sin(θ)/(8*sqrt(pi)*α_0)

u_211(r,θ,φ,Z)*


-Z*r*sqrt(Z**3/α_0**3)*exp(-1.0*I*φ)*exp(-Z*r/(2*α_0))*sin(θ)/(8*sqrt(pi)*α_0)

Third, we have the state **|310>**$=u_{310}(r,θ,φ,Ζ)=R_{31}(r,Ζ)*Y^0_1(θ,φ)$:

In [12]:
# State u_310 = R_31*Y_10 of hydrogeneous atoms

# Numerical definition
def u_310(r,θ,φ,Z):
    return R_31(r,Z)*Y_10(θ,φ)

# Symbolic definitions
def u_310_sym(r,θ,φ,Z):
    return R_nl_sym(r,3,1,Z)*Y_10_sym(θ,φ)

def u_310_star_sym(r,θ,φ,Z):
    return R_nl_sym(r,3,1,Z)*Y_10_star_sym(θ,φ)

# Printing formula of the state
print("u_310(r,θ,φ,Z)")
display(u_310_sym(r,theta,phi,Zeta))

# Printing formula of the state's conjugate
print("u_310(r,θ,φ,Z)*")
display(u_310_star_sym(r,theta,phi,Zeta))

u_310(r,θ,φ,Z)


sqrt(2)*Z*r*sqrt(Z**3/α_0**3)*(-2*Z*r/(3*α_0) + 4)*exp(-Z*r/(3*α_0))*cos(θ)/(54*sqrt(pi)*α_0)

u_310(r,θ,φ,Z)*


sqrt(2)*Z*r*sqrt(Z**3/α_0**3)*(-2*Z*r/(3*α_0) + 4)*exp(-Z*r/(3*α_0))*cos(θ)/(54*sqrt(pi)*α_0)

And at last, we have the state **|311>**$=u_{311}(r,θ,φ,Ζ)=R_{31}(r,Ζ)*Y^1_1(θ,φ)$:

In [13]:
# State u_311 = R_31*Y_11 of hydrogeneous atoms

# Numerical definition
def u_311(r,θ,φ,Z):
    return R_31(r,Z)*Y_11(θ,φ)

# Symbolic definitions
def u_311_sym(r,θ,φ,Z):
    return R_nl_sym(r,3,1,Z)*Y_11_sym(θ,φ)

def u_311_star_sym(r,θ,φ,Z):
    return R_nl_sym(r,3,1,Z)*Y_11_star_sym(θ,φ)

# Printing formula of the state
print("u_311(r,θ,φ,Z)")
display(u_311_sym(r,theta,phi,Zeta))

# Printing formula of the state's conjugate
print("u_311(r,θ,φ,Z)*")
display(u_311_star_sym(r,theta,phi,Zeta))

u_311(r,θ,φ,Z)


-Z*r*sqrt(Z**3/α_0**3)*(-2*Z*r/(3*α_0) + 4)*exp(1.0*I*φ)*exp(-Z*r/(3*α_0))*sin(θ)/(54*sqrt(pi)*α_0)

u_311(r,θ,φ,Z)*


-Z*r*sqrt(Z**3/α_0**3)*(-2*Z*r/(3*α_0) + 4)*exp(-1.0*I*φ)*exp(-Z*r/(3*α_0))*sin(θ)/(54*sqrt(pi)*α_0)

## 2. Calculating the requested dot products

**A. General form of the products in spherical coordinates**<br>
Let's discuss now, the methodology for the calculation of the requested dot products. The general form of these products is:
$$I=<nlm|\hat{A}|n'l'm'>$$
or in integration form
$$I=\int_V u^*_{nlm}(r,θ,φ,Z)\hat{A}u_{n'l'm'}(r,θ,φ,Z)dV\space\space[2.1]$$
where $\hat{A}$ is an operator (in our case $\hat{A}=\hat{x}$ or $\hat{A}=\hat{y}$) and $V$ is the **3D** Εuclidean integration space.
Using spherical coordinates, the differential of this space becomes:
$$dV=r^2\sinθdθdφdr$$
where $r$ is the radius, $\theta$ is the polar angle and $\phi$ is the azimuthal angle. Therefore, the integral of $[2.1]$, is basically a triple integral:
$$I=\int^{\infty}_{r=0}\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}u^*_{nlm}(r,θ,φ,Z)\hat{A}u_{n'l'm'}(r,θ,φ,Z)r^2\sinθdθdφdr\space\space[2.2]$$

**B. Separating the radial and angular parts of the triple integral**<br>
To address the triple integration of $[2.2]$, we write the states $u^*_{nlm}$ and $u_{n'l'm'}$ as products of their radial and angular components: 
$$u^*_{nlm}(r,\theta,\phi,Z)=[R_{nl}(r,Z)Y^m_l(\theta,\phi)]^*=Y^m_l(\theta,\phi)^*R_{nl}(r,Z)^*\space\space [2.3]$$
and
$$u_{n'l'm'}(r,\theta,\phi,Z)=R_{n'l'}(r,Z)Y^{m'}_{l'}(\theta,\phi)\space\space [2.4]$$
As for the operators $\hat{x}$ and $\hat{y}$, we have:
$$\hat{x}=x=r\sin\theta\cos\phi \space\space [2.5]$$
and
$$\hat{y}=y=r\sin\theta\sin\phi \space\space [2.6]$$
By replacing, the equations $[2.3]$ and $[2.4]$ to equation $[2.2]$, we get:
$$I=\int^{\infty}_{r=0}\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}Y^m_l(\theta,\phi)^*R_{nl}(r,Z)^*\hat{A}R_{n'l'}(r,Z)Y^{m'}_{l'}(\theta,\phi)r^2\sinθdθdφdr \space\space[2.7]$$
and if we set $\hat{A}=\hat{x}$ from $[2.5]$ in $[2.7]$, we get:
$$I(\hat{A}=\hat{x})=I_x=\int^{\infty}_{r=0}\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}Y^m_l(\theta,\phi)^*R_{nl}(r,Z)^*r\sinθ\cosφ R_{n'l'}(r,Z)Y^{m'}_{l'}(\theta,\phi)r^2\sinθdθdφdr \space\space[2.8]$$
Now, we can separate the radial and the angular parts in the integral of $[2.8]$ and get:
$$I_x=\int^{\infty}_{r=0}R_{nl}(r,Z)^*R_{n'l'}(r,Z)r^3dr*\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}Y^m_l(\theta,\phi)^*Y^{m'}_{l'}(\theta,\phi)\sin^2θ\cosφdθdφ$$
or
$$I_x=I_{Radial_x}*I_{Angular_x}$$
where $I_{Radial_x}$ is the radial integral:
$$I_{Radial_x}=\int^{\infty}_{r=0}R_{nl}(r,Z)^*R_{n'l'}(r,Z)r^3dr \space\space [2.9]$$
and $I_{Angular_x}$ is the angular integral:
$$I_{Angular_x}=\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}Y^m_l(\theta,\phi)^*Y^{m'}_{l'}(\theta,\phi)\sin^2θ\cosφdθdφ \space\space [2.10]$$

In the same way, if we replace $\hat{A}=\hat{y}$ from $[2.6]$ in $[2.7]$ and separate the radial and angular parts, we result in the following radial integral $I_{Radial_y}$:
$$I_{Radial_y}=\int^{\infty}_{r=0}R_{nl}(r,Z)^*R_{n'l'}(r,Z)r^3dr \space\space [2.11]$$
and the following angular integral $I_{Angular_y}$:
$$I_{Angular_y}=\int^{2\pi}_{\phi=0}\int^{\pi}_{\theta=0}Y^m_l(\theta,\phi)^*Y^{m'}_{l'}(\theta,\phi)\sin^2θ\sinφdθdφ \space\space [2.12]$$

As we can see, the contribution of the two operators $\hat{x}$ and $\hat{y}$ differs only in the angular integral.

**C. Calculating the radial and the angular integrals**<br>
Since complex numbers are involved in the integration process and the integration is over real numbers ($r,θ,φ$), we can define the integrand function $f$ of the integrals $[2.9]$,$[2.10]$,$[2.11]$ and $[2.12]$ above, as:
$$f=f_{Real}+i*f_{Imaginary}$$
which results in the following form for the respective integral:
$$I=\int_{S}fdS=\int_{S}(f_{Real}+i*f_{Imaginary})dS=I^{Real}+i*I^{Imaginary}$$
where:
$$I^{Real}=\int_{S}f_{Real}dS$$
and
$$I^{Imaginary}=\int_{S}f_{Imaginary}dS$$
and $S$ is the integration space.

Thus, for the total triple integral $I_x$ we have:
$$I_x=I_{Radial_x}*I_{Angular_x}=[I^{Real}_{Radial_x}+i*I^{Imaginary}_{Radial_x}]*[I^{Real}_{Angular_x}+i*I^{Imaginary}_{Angular_x}]$$
while for the total triple integral $I_y$ we have:
$$I_y=I_{Radial_y}*I_{Angular_y}=[I^{Real}_{Radial_y}+i*I^{Imaginary}_{Radial_y}]*[I^{Real}_{Angular_y}+i*I^{Imaginary}_{Angular_y}]$$

This method is implemented below for the calculation of the requested dot products, setting $Z=1$ for the hydrogen atom.

### **2.1 Dot product $<210|x|311>$**

In [14]:
# Dot product <210|x|311>

# x = r*sinθ*cosφ
# r: included in radial part's integral
# sinθ*cosφ: included in angular part's integral

print("> Calculating the dot product:")
print("<210|x|311>\n")

# Separate calculate the radial and angular integrals
print("> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART")
print("-----------------------------------------------------")

# Radial part's integral
def f_R(r):
    return np.conj(R_21(r,1))*R_31(r,1)*r**3
I_R_real = quad(lambda r: np.real(f_R(r)),0,np.Infinity)[0]
I_R_imag = quad(lambda r: np.imag(f_R(r)),0,np.Infinity)[0]
I_R = I_R_real + 1j*I_R_imag
print(">> Radial part's integral I_Rad:")
print(I_R)

# Angular part's integral
def f_Ang(θ,φ):
    return np.conj(Y_10(θ,φ))*Y_11(θ,φ)*np.sin(θ)**2*np.cos(φ)
I_Ang_real = dblquad(lambda θ,φ: np.real(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang_imag = dblquad(lambda θ,φ: np.imag(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang = I_Ang_real + 1j*I_Ang_imag
print("\n>> Angular part's integral I_Ang:")
print(I_Ang)

result = I_R*I_Ang
print("\n>> Dot Product Result I_Rad*I_Ang:")
print(result)
print("-----------------------------------------------------")

> Calculating the dot product:
<210|x|311>

> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART
-----------------------------------------------------
>> Radial part's integral I_Rad:
(-0.09360506880000004+0j)

>> Angular part's integral I_Ang:
(-3.067787304177841e-17-4.509068613625614e-19j)

>> Dot Product Result I_Rad*I_Ang:
(2.8716044167133347e-18+4.2207167780234635e-20j)
-----------------------------------------------------


### **2.2 Dot product $<211|x|310>$**

In [15]:
# Dot product <211|x|310>

# x = r*sinθ*cosφ
# r: included in radial part's integral
# sinθ*cosφ: included in angular part's integral

print("> Calculating the dot product:")
print("<211|x|310>\n")

# Separate calculate the radial and angular integrals
print("> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART")
print("-----------------------------------------------------")

# Radial part's integral
def f_R(r):
    return np.conj(R_21(r,1))*R_31(r,1)*r**3
I_R_real = quad(lambda r: np.real(f_R(r)),0,np.Infinity)[0]
I_R_imag = quad(lambda r: np.imag(f_R(r)),0,np.Infinity)[0]
I_R = I_R_real + 1j*I_R_imag
print(">> Radial part's integral I_Rad:")
print(I_R)

# Angular part's integral
def f_Ang(θ,φ):
    return np.conj(Y_11(θ,φ))*Y_10(θ,φ)*np.sin(θ)**2*np.cos(φ)
I_Ang_real = dblquad(lambda θ,φ: np.real(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang_imag = dblquad(lambda θ,φ: np.imag(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang = I_Ang_real + 1j*I_Ang_imag
print("\n>> Angular part's integral I_Ang:")
print(I_Ang)

result = I_R*I_Ang
print("\n>> Dot Product Result I_Rad*I_Ang:")
print(result)
print("-----------------------------------------------------")

> Calculating the dot product:
<211|x|310>

> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART
-----------------------------------------------------
>> Radial part's integral I_Rad:
(-0.09360506880000004+0j)

>> Angular part's integral I_Ang:
(-3.067787304177841e-17+4.509068613625614e-19j)

>> Dot Product Result I_Rad*I_Ang:
(2.8716044167133347e-18-4.2207167780234635e-20j)
-----------------------------------------------------


### **2.3 Dot product $<210|y|311>$**

In [16]:
# Dot product <210|y|311>

# y = r*sinθ*sinφ
# r: included in radial part's integral
# sinθ*sinφ: included in angular part's integral

print("> Calculating the dot product:")
print("<210|y|311>\n")

# Separate calculate the radial and angular integrals
print("> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART")
print("-----------------------------------------------------")

# Radial part's integral
def f_R(r):
    return np.conj(R_21(r,1))*R_31(r,1)*r**3
I_R_real = quad(lambda r: np.real(f_R(r)),0,np.Infinity)[0]
I_R_imag = quad(lambda r: np.imag(f_R(r)),0,np.Infinity)[0]
I_R = I_R_real + 1j*I_R_imag
print(">> Radial part's integral I_Rad:")
print(I_R)

# Angular part's integral
def f_Ang(θ,φ):
    return np.conj(Y_10(θ,φ))*Y_11(θ,φ)*np.sin(θ)**2*np.sin(φ)
I_Ang_real = dblquad(lambda θ,φ: np.real(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang_imag = dblquad(lambda θ,φ: np.imag(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang = I_Ang_real + 1j*I_Ang_imag
print("\n>> Angular part's integral I_Ang:")
print(I_Ang)

result = I_R*I_Ang
print("\n>> Dot Product Result I_Rad*I_Ang:")
print(result)
print("-----------------------------------------------------")

> Calculating the dot product:
<210|y|311>

> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART
-----------------------------------------------------
>> Radial part's integral I_Rad:
(-0.09360506880000004+0j)

>> Angular part's integral I_Ang:
(-8.069487063086716e-19-2.936095568006275e-17j)

>> Dot Product Result I_Rad*I_Ang:
(7.553448917209423e-20+2.7483342764660254e-18j)
-----------------------------------------------------


### **2.4 Dot product $<211|y|310>$**

In [17]:
# Dot product <211|y|310>

# y = r*sinθ*sinφ
# r: included in radial part's integral
# sinθ*sinφ: included in angular part's integral

print("> Calculating the dot product:")
print("<211|y|310>\n")

# Separate calculate the radial and angular integrals
print("> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART")
print("-----------------------------------------------------")

# Radial part's integral
def f_R(r):
    return np.conj(R_21(r,1))*R_31(r,1)*r**3
I_R_real = quad(lambda r: np.real(f_R(r)),0,np.Infinity)[0]
I_R_imag = quad(lambda r: np.imag(f_R(r)),0,np.Infinity)[0]
I_R = I_R_real + 1j*I_R_imag
print(">> Radial part's integral I_Rad:")
print(I_R)

# Angular part's integral
def f_Ang(θ,φ):
    return np.conj(Y_11(θ,φ))*Y_10(θ,φ)*np.sin(θ)**2*np.sin(φ)
I_Ang_real = dblquad(lambda θ,φ: np.real(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang_imag = dblquad(lambda θ,φ: np.imag(f_Ang(θ,φ)),0,2*np.pi,0,np.pi)[0]
I_Ang = I_Ang_real + 1j*I_Ang_imag
print("\n>> Angular part's integral I_Ang:")
print(I_Ang)

result = I_R*I_Ang
print("\n>> Dot Product Result I_Rad*I_Ang:")
print(result)
print("-----------------------------------------------------")

> Calculating the dot product:
<211|y|310>

> SEPARATE CALCULATION OF THE RADIAL AND ANGULAR PART
-----------------------------------------------------
>> Radial part's integral I_Rad:
(-0.09360506880000004+0j)

>> Angular part's integral I_Ang:
(-8.069487063086716e-19+2.936095568006275e-17j)

>> Dot Product Result I_Rad*I_Ang:
(7.553448917209423e-20-2.7483342764660254e-18j)
-----------------------------------------------------
